In [1]:
import os
import re
import json
import random
import codecs
from template_config import *
from nltk import word_tokenize
from collections import defaultdict
from transformers.tokenization_roberta import RobertaTokenizer

SEP_TOKEN = "</s>"
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
MAX_TOKEN_LEN = 189

To use data.metrics please install scikit-learn. See https://scikit-learn.org/stable/index.html


### Read tables from text files

In [2]:
train_corpus = "data/wikitable_dup1_row1.txt"
# output_file = "data/data_comb_tables.txt"

In [3]:
def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

def check_name(inpStr):
    return len(inpStr) > 1 and "-" not in inpStr and not hasNumbers(inpStr)

def gen_name(title, must_have=False):
    title_tokens = title.split(" ")
    qualify_words = []
    for w in title_tokens:
        if check_name(w):
            qualify_words.append(w)
    
    if random.random() < 0.4:
        name = " ".join(qualify_words[-2:])
    else:
        name = " ".join(qualify_words[-1:])
    
    if name != "":
        return name
    
    if must_have:
        return title_tokens[0]
    else:
        return name

In [4]:
def main_process(train_corpus):
    total_count = 0
    webtables = []
    with open(train_corpus, "r", encoding="utf-8") as f:
        for line in f:
            skip = False
            if total_count % 100000 == 0:
                print("processed: ", total_count)
            tokens = line.lower().replace("<special7>", "<tabn>").replace("<special8>", "<coln>").replace("<special9>", "<entry>").replace("*", "").replace("|||", "")
            table = {"columns": [], "values": [], "columns_original": [], "column_types": []}
            chunks = tokens.split(" <coln> ")
            for chunk in chunks:
                if "<tabn>" in chunk:
                    page_title = chunk.replace("<tabn>", "").strip()
                    table_name = gen_name(page_title)
                    table["name"] = table_name
                    if table_name == "" or len(table_name) < 2:
                        skip = True
                else:
                    assert "<entry>" in chunk
                    chunk_toks = chunk.split(" <entry> ")
                    if len(chunk_toks) == 2:
                        col_name, entry = chunk_toks[0].strip(), chunk_toks[1].strip()
                        if len(col_name) > 1:
                            table["columns"].append(" ".join(col_name.split(" ")[:5]))
                            table["columns_original"].append(col_name)
                            ctype = "text"
                            if entry.isdigit():
                                ctype = "real"
                            table["column_types"].append(ctype)
                            table["values"].append(" ".join(entry.split(" ")[:5]))
                            
            
            if len(table["columns"]) < 3:
                skip = True
            if not skip:
                table_name = table["name"]
                table["columns"] = [table_name + " *"] + table["columns"]
                table["columns_original"] = ["*"] + table["columns_original"]
                table["column_types"] = ["text"] + table["column_types"]
                table["values"] = ["all"] + table["values"]
                tabn_str = "_".join(table_name.split(" "))
                table["columns"] = [tabn_str +" "+ hd for hd in table["columns"]]
                if "*" not in table['columns'][0]:
                    print(table['columns'])
                webtables.append(table)
                
            total_count += 1
            
    return webtables

In [5]:
web_tables = main_process(train_corpus)

processed:  0
processed:  100000
processed:  200000


### Read NL-SQL templates and sql component mapping file

In [6]:
MAX_COL_NUM = 25
OPS = ["=", ">", "<", ">=", "<=", "!=", "LIKE"]
nlsql_templates_file = "data/nlsql_templates.txt"
spider_data_file = '/data/projects/nl2sql/datasets_final/data/'
sql_components_file = "data/sql_components.json"

In [7]:
# read NL-SQL templates
templates = []
with open(nlsql_templates_file) as fp:
    lines = fp.readlines()
    template_one = {}
    for line in lines:
        if "\n" == line:
            templates.append(template_one) 
        elif "SQL Pattern:" in line:
            template_one = {}
            sps = line.strip().replace("SQL Pattern: ", "").split("|||")
            template_one["questions"] = []
            if len(sps) == 1:
                template_one["SQL pattern"] = sps[0]
                template_one["SQL constraints"] = []
            elif len(sps) == 2:
                template_one["SQL pattern"] = sps[0]
                template_one["SQL constraints"] = [x.strip() for x in sps[1].split("|") if x != " "]
            else:
                print("\n======Error warning!!!!")
        elif "count: " in line:
            sql_count = int(line.strip().replace("count: ", ""))
            template_one["count"] = sql_count
        elif "question:  " in line:
            sps = line.strip().replace("question:  ", "").split("|||")
            question = sps[0]
            if len(sps) == 2:
                q_constraints = [x.strip() for x in sps[1].split("|") if x != " "]
            else:
                q_constraints = []
            template_one["questions"].append((question, q_constraints))

In [8]:
templates_one_table = []
for template in templates:
    sql_constraints = template['SQL constraints']
    sql_pattern = template["SQL pattern"]
    questions = template["questions"]
    skip = False
    for constraint in sql_constraints:
        if "id" in constraint or "T1" in constraint:
            skip = True
    questions_after = []     
    if not skip:
        for q, qc in questions:
            if "TABLE1" not in q:
                questions_after.append((q, qc))
        if len(questions_after) > 0:
            template_one = {}
            template_one['SQL constraints'] = sql_constraints
            template_one['SQL pattern'] = sql_pattern
            template_one["questions"] = questions_after
            templates_one_table.append(template_one)

In [9]:
all_constraints = []
for tmp in templates_one_table:
    all_constraints.extend(tmp['SQL constraints'])
    for q in tmp['questions']:
        all_constraints.extend(q[1])

print(list(set(all_constraints)))

['{AGG0}=MIN', 'P0=<', 'P1==', 'P0=>', 'P0=P1==', 'P0==', '{AGG0}=MAX', 'P0=P1=P2==']


In [10]:
# read SQL component file
with open(sql_components_file) as json_file:
    sql_components = json.load(json_file)

### Unify and combine tables as databases

In [12]:
def create_dbs(tables):
#     random.shuffle(tables)  
    dbs = []
    cur_cols = []
    db_one = []
    ahd_cols = []
    for i, tab in enumerate(tables):
        if i % 100000 == 0:
            print("processed: ", i)
        if len(db_one) <= random.choice([0, 1]) and len(ahd_cols) < MAX_COL_NUM:
            db_one.append(tab)
            cur_cols.extend([col+"."+tab["name"] for col in tab["columns"]])
            if i+1 < len(tables):
                ahd_cols = cur_cols + [col+"."+tables[i+1]["name"] for col in tables[i+1]["columns"]]
            else:
                 break
        else:
            if len(cur_cols) == len(list(set(cur_cols))):
                if len(db_one) > 1:
                    db_one_new = []
                    for tab in db_one:
                        if tab["columns"][0] == "id":
                            tab["columns"] = tab["columns"][1:]
                            tab["column_types"] = tab["column_types"][1:]
                            tab["columns_original"] = tab["columns_original"][1:]
                            tab["values"] = tab["values"][1:]
                            
                        if random.random() < 0.7:
                            index_col = "id"
                            if random.random() < 0.3:
                                index_col = "name"

                            if index_col not in tab["columns"]:
                                tabn_str = "_".join(tab["name"].split(" "))
                                tab["columns"] = [tab["columns"][0]] + [tabn_str +" "+ index_col] + tab["columns"][1:]
                                val_add = 1
                                if index_col == "name":
                                    val_add = "value"
                                tab["values"] = [tab["values"][0]] + [val_add] + tab["values"][1:]
                                tab["column_types"] = [tab["column_types"][0]] + ["text"] + tab["column_types"][1:]
                                tab["columns_original"] = [tab["columns_original"][0]] + [index_col] + tab["columns_original"][1:]
                        db_one_new.append(tab)
                    dbs.append(db_one_new)
                else:
                    dbs.append(db_one)
            db_one = []
            cur_cols = []
            ahd_cols = []
            
    return dbs

In [13]:
web_tables[5]

{'columns': ['cup_team cup team *',
  'cup_team player',
  'cup_team years played',
  'cup_team total w-l',
  'cup_team singles w-l',
  'cup_team doubles w-l'],
 'values': ['all', 'cho soong-jae ( 110 )', '1 ( 1011 )', '1–0', '1–0', '1–0'],
 'columns_original': ['*',
  'player',
  'years played',
  'total w-l',
  'singles w-l',
  'doubles w-l'],
 'column_types': ['text', 'text', 'text', 'text', 'text', 'text'],
 'name': 'cup team'}

In [14]:
webtable_dbs = create_dbs(web_tables)

processed:  0
processed:  100000
processed:  200000


In [15]:
len(webtable_dbs)

83701

In [16]:
for db in webtable_dbs[:1000]:
    tab_names = []
    col_count = 0
    for tab in db:
        tab_names.append(tab["name"])
        col_count += len(tab["columns"])
    print("----------")
    print("table names: ", tab_names)
    print("column num: ", col_count)
    print("table num: ", len(tab_names))

----------
table names:  ['loading', 'transit']
column num:  15
table num:  2
----------
table names:  ['switzerland']
column num:  8
table num:  1
----------
table names:  ['cup team', 'award']
column num:  16
table num:  2
----------
table names:  ['fraternities sororities']
column num:  7
table num:  1
----------
table names:  ['league', 'girlfriends']
column num:  17
table num:  2
----------
table names:  ['speakers gaeltacht']
column num:  6
table num:  1
----------
table names:  ['terry labonte', 'song contest']
column num:  22
table num:  2
----------
table names:  ['lattice constants']
column num:  6
table num:  1
----------
table names:  ['rail class', 'ya']
column num:  15
table num:  2
----------
table names:  ['iroc xx']
column num:  6
table num:  1
----------
table names:  ['stozhary stozhary']
column num:  6
table num:  1
----------
table names:  ['conductive film']
column num:  6
table num:  1
----------
table names:  ['zina', 'house']
column num:  19
table num:  2
-----

column num:  17
table num:  2
----------
table names:  ['expeditions', 'envirothon']
column num:  18
table num:  2
----------
table names:  ['detroit rockers', 'division league']
column num:  16
table num:  2
----------
table names:  ['longer']
column num:  6
table num:  1
----------
table names:  ['bagua', 'cup']
column num:  15
table num:  2
----------
table names:  ['satellite', 'apostolic church']
column num:  18
table num:  2
----------
table names:  ['midwest region']
column num:  8
table num:  1
----------
table names:  ['league', 'conflicts']
column num:  20
table num:  2
----------
table names:  ['moldova', 'mgp nordic']
column num:  14
table num:  2
----------
table names:  ['moldova', 'encodings teletext']
column num:  16
table num:  2
----------
table names:  ['montfortian dialect', 'cup']
column num:  16
table num:  2
----------
table names:  ['khor municipality']
column num:  11
table num:  1
----------
table names:  ['prix']
column num:  9
table num:  1
----------
table 

column num:  6
table num:  1
----------
table names:  ['george utley']
column num:  8
table num:  1
----------
table names:  ['ski center']
column num:  6
table num:  1
----------
table names:  ['mori']
column num:  6
table num:  1
----------
table names:  ['david rikl']
column num:  21
table num:  1
----------
table names:  ['title', 'club']
column num:  14
table num:  2
----------
table names:  ['ghana', 'championships']
column num:  15
table num:  2
----------
table names:  ['lukoil']
column num:  6
table num:  1
----------
table names:  ['zina garrison', 'morozova']
column num:  23
table num:  2
----------
table names:  ['house', 'in china']
column num:  17
table num:  2
----------
table names:  ['portugal']
column num:  6
table num:  1
----------
table names:  ['technology alumni', 'cup team']
column num:  16
table num:  2
----------
table names:  ['cup team', 'team']
column num:  20
table num:  2
----------
table names:  ['series dams', 'team']
column num:  18
table num:  2
-----

### Start generate NL-SQL examples based on new databases and CFG grammars

##### detect question and SQL slots and process constraints

In [17]:
def get_sql_slots(sql_pattern):
    sql_tokens = sql_pattern.split(" ")
    columns = {}
    ops = {}
    values = {}
    aggs = {}
    dasc = False
    slots = []
    val_pros = []
    for i, tok in enumerate(sql_tokens):
        if "{" in tok and "}" in tok and "FROM" not in tok:
            if tok not in slots:
                slots.append(tok)
                
        if "AGG" in tok:
            if i + 2 < len(sql_tokens) and "(" == sql_tokens[i+1]:
                if "COLUMN" in sql_tokens[i+2]:
                    if sql_tokens[i+2] not in columns.keys():
                        columns[sql_tokens[i+2]] = ["number"]
                    else:
                        columns[sql_tokens[i+2]].append("number")
                    aggs[tok] = sql_tokens[i+2]
                else:
                    print("\nTemplate Error: AGG format is wrong!!!")
                    print(sql_pattern)
        elif "COLUMN" in tok:
            if tok not in columns.keys():
                columns[tok] = []
        elif "OP" in tok:
            if i - 1 >= 0 and "COLUMN" in sql_tokens[i-1]:
                ops[tok] = [sql_tokens[i-1]]
                if i + 1 < len(sql_tokens) and "VALUE" in sql_tokens[i+1]:
                    ops[tok].append(sql_tokens[i+1])
                    val_pros.append(sql_tokens[i+1])
            elif i - 2 >= 0 and ")" == sql_tokens[i-1] and ("COLUMN" in sql_tokens[i-2] or "*" == sql_tokens[i-2]):
                ops[tok] = [sql_tokens[i-2]]
                if i + 1 < len(sql_tokens) and "VALUE" in sql_tokens[i+1]:
                    ops[tok].append(sql_tokens[i+1])
                    val_pros.append(sql_tokens[i+1])
            else:
                print("\nTemplate Error: OP format is wrong!!!")
                print(sql_pattern)
        elif "VALUE" in tok and tok not in val_pros:
            """
            OP} {VALUE0}
            LIMIT {VALUE0}
            {COLUMN1} BETWEEN {VALUE0} AND {VALUE1}
            HAVING COUNT ( * ) {OP1} {VALUE1}
            = {VALUE1}
            """
            if i - 2 >= 0 and ("BETWEEN" == sql_tokens[i-1] or "AND" == sql_tokens[i-1]):
                values[tok] = "number"
                if "BETWEEN" == sql_tokens[i-1]:
                    columns[sql_tokens[i-2]].append("number")
            elif i - 1 >= 0 and "LIMIT" == sql_tokens[i-1]:
                values[tok] = "integer"
            elif i - 1 >= 0 and "=" == sql_tokens[i-1]:
                assert "COLUMN" in sql_tokens[i-2]
                columns[sql_tokens[i-2]].append(tok)
            else:
                print("\nTemplate Error: VALUE format is wrong!!!")
                print(sql_pattern)
        elif "DASC" in tok:
            dasc = True
    
    return (list(set(slots)), columns, ops, values, aggs, dasc)


def get_q_slots(question):
    q_toks = question.strip().split(" ")
    q_slots = list(set([tok for tok in q_toks if "TABLE" in tok or "SC" in tok or ("{" in tok and "}" in tok)]))
    
    return q_slots
    

def process_constraints(constraints, columns, slots):
    slot_values = {}
    skip_db_with_one_table = False
    for constraint in constraints:
        if "P0==" == constraint:
            assert "{OP0}" in slots
            slot_values["{OP0}"] = "="
        elif "P1==" == constraint:
            assert "{OP1}" in slots
            slot_values["{OP1}"] = "="
        elif "P0=P1==" == constraint:
            assert "{OP0}" in slots and "{OP1}" in slots
            slot_values["{OP0}"] = "="
            slot_values["{OP1}"] = "="
        elif "P0=P1=P2==" == constraint:
            assert "{OP0}" in slots and "{OP1}" in slots and "{OP2}" in slots
            slot_values["{OP0}"] = "="
            slot_values["{OP1}"] = "="
            slot_values["{OP2}"] = "="
        elif "P0=>" == constraint:
            assert "{OP0}" in slots
            slot_values["{OP0}"] = ">"
        elif "P0=<" == constraint:
            assert "{OP0}" in slots
            slot_values["{OP0}"] = "<"
        elif "{AGG0}=MIN" == constraint:
            assert "{AGG0}" in slots
            slot_values["{AGG0}"] = "MIN"
        elif "{AGG0}=MAX" == constraint:
            assert "{AGG0}" in slots
            slot_values["{AGG0}"] = "MAX"
        elif "C0-id" == constraint:
            skip_db_with_one_table = True
            assert "{COLUMN0}" in slots and "{COLUMN0}" in columns.keys()
            columns["{COLUMN0}"].append("id")
        elif "C1-id" == constraint:
            skip_db_with_one_table = True
            assert "{COLUMN1}" in slots and "{COLUMN1}" in columns.keys()
            columns["{COLUMN1}"].append("id")
        elif "C2-id" == constraint:
            skip_db_with_one_table = True
            assert "{COLUMN2}" in slots and "{COLUMN2}" in columns.keys()
            columns["{COLUMN2}"].append("id")
        elif "C3-T1" == constraint:
            skip_db_with_one_table = True
            assert "{COLUMN3}" in slots and "{COLUMN3}" in columns.keys()
            columns["{COLUMN3}"].append("T1")
        elif "T0-T1-JOIN" == constraint or 'T0-T1-NO-JOIN' == constraint:
            skip_db_with_one_table = True
            columns["{COLUMN0}"].append("T0")
            if "{COLUMN1}" in columns.keys():
                columns["{COLUMN1}"].append("T1")
    
    return (slot_values, columns, skip_db_with_one_table)


# helper function
def gen_col_info(col_str, columns, columns_inf):
    col_conds = columns[col_str]
    value_slot = [cc for cc in col_conds if "VALUE" in cc]
    col = ""
    value_val = None
    if "id" in col_conds:
        has_id = False
        for c, t, v in columns_inf:
            if "id" in col or "name" in col:
                has_id = True
                col, ctype, values = c, t, v
                break
        if not has_id:
            col, ctype, value = columns_inf[0]
    elif "number" in col_conds:
        for colinfo in columns_inf[1:]:
            if colinfo[1] == "real":
                col, ctype, value = colinfo
    if col == "":
        col, ctype, value = random.choice(columns_inf[1:])

    if len(value_slot) > 0:
        assert len(value_slot) < 3
        if len(value_slot) == 1:
            value_val = [(value_slot[0], value)]
        else:
            value_val = [(value_slot[0], value), (value_slot[1], value)]
    
    return (col, value_val)


def replace_dict(inp, dicts):
    for rep_in, rep_out in dicts.items():
        inp = inp.replace(rep_in, str(rep_out))
    
    return inp


##### Get classification label for each column based on SQL templates

In [18]:
STRUCT_KEYWORDS = ["WHERE", "GROUP_BY", "HAVING", "ORDER_BY", "SELECT"]
EXTRA_OPS = ["NOT_IN", "IN", "BETWEEN", "="]
COUNT = "COUNT"
OTHER_KEYWORDS = ["LIMIT"] #AGG, OP, DASC, OR, =
NEST_KEYWORDS = ["EXCEPT", "UNION", "INTERSECT"]

def get_labels(sql_pattern):
    sql_tokens = sql_pattern.replace("GROUP BY", "GROUP_BY").replace("ORDER BY", "ORDER_BY").replace("NOT IN", "NOT_IN").split(" ")
    columns = {}
    cur_nest = ""
    cur_struct = ""
    cur_len = len(sql_tokens)
    select_count = 0
    for i, tok in enumerate(sql_tokens):
        if tok in NEST_KEYWORDS:
            if cur_nest == "" or cur_nest == "OP_SEL":
                cur_nest = tok
            else:
                cur_nest = cur_nest + " " + tok
        elif tok in STRUCT_KEYWORDS:
            cur_struct = tok
            if tok == "SELECT":
                select_count += 1
                if select_count > 1 and cur_nest == "":
                    cur_nest = "OP_SEL"
        elif "COLUMN" in tok or "*" == tok:
            if tok not in columns.keys():
                columns[tok] = []
            # SELECT {COLUMN0}
            # SELECT {COLUMN0} , {COLUMN1}
            # SELECT {AGG0} ( {COLUMN0} )
            # SELECT {COLUMN0} {FROM} WHERE {COLUMN1} {OP} ( SELECT {AGG0} ( {COLUMN1} ) {FROM} ) AND {COLUMN2} {OP0} {VALUE0}
            if cur_struct == "SELECT":
                if "," == sql_tokens[i-1] or "SELECT" == sql_tokens[i-1]:
                    columns[tok].append(cur_nest + " " + cur_struct)
                elif "(" == sql_tokens[i-1]:
                    columns[tok].append(cur_nest + " " + cur_struct + " " + sql_tokens[i-2])
                else:
                    print("\nWarning: unexcepted SELECT format")
                    print(sql_pattern)
            # WHERE {COLUMN} {OP}
            # WHERE {COLUMN2} {OP0}
            # WHERE OR {COLUMN2} {OP0}
            # WHERE {COLUMN2} BETWEEN
            elif cur_struct == "WHERE":
                assert "OP" in sql_tokens[i+1] or sql_tokens[i+1] in EXTRA_OPS
                last_tok = sql_tokens[i-1]
                if "OR" == last_tok or (i+3 < cur_len and "OR" == sql_tokens[i+3]):
                    columns[tok].append(cur_nest + " " + cur_struct + " OR " + sql_tokens[i+1])
                elif "WHERE" == last_tok or "AND" == last_tok:
                    columns[tok].append(cur_nest + " " + cur_struct + " " + sql_tokens[i+1])
                else:
                    print("\nWarning: unexcepted WHERE format")
            # GROUP BY {COLUMN0} , {COLUMN0}
            elif cur_struct == "GROUP_BY":
                columns[tok].append(cur_nest + " " + cur_struct)
            # HAVING COUNT ( * ) {OP0}
            # HAVING {AGG0} ( {COLUMN2} ) {OP0}
            elif cur_struct == "HAVING":
                last_tok = sql_tokens[i-1]
                if last_tok != "(" and not ("AGG" in sql_tokens[i-2] or COUNT == sql_tokens[i-2]):
                    print("\nWarning: unexcepted HAVING format")
                columns[tok].append(cur_nest + " " + cur_struct + " " + sql_tokens[i-2] + " " + sql_tokens[i+2])
            # ORDER BY COUNT ( * ) {DASC} LIMIT
            # ORDER BY COUNT ( * ) {DASC}
            # ORDER BY {COLUMN1} {DASC} LIMIT
            # ORDER BY {COLUMN1} LIMIT
            # ORDER BY {COLUMN1} , {COLUMN1} {DASC} LIMIT
            # ORDER BY {COLUMN1} {DASC} if no DASC then is ASC
            elif cur_struct == "ORDER_BY":
                last_tok = sql_tokens[i-1]
                if last_tok == "(":
                    dasc_tok = "{DASC}"
                    limit_tok = ""
                    if sql_tokens[i+2] != "{DASC}":
                        dasc_tok = "ASC"
                        if sql_tokens[i+2] == "LIMIT":
                            limit_tok = "LIMIT"
                    elif i+3 < cur_len and sql_tokens[i+3] == "LIMIT":
                        limit_tok = "LIMIT"
                        
                    columns[tok].append(cur_nest + " " + cur_struct + " " + sql_tokens[i-2] + " " + dasc_tok + " " + limit_tok)
                elif last_tok == "ORDER_BY" or last_tok == ",":
                    dasc_tok = "ASC"
                    limit_tok = ""
                    # small dirty pass
                    if i+1 < cur_len and sql_tokens[i+1] == "{DASC}":
                        dasc_tok = "{DASC}"
                        if i+2 < cur_len and sql_tokens[i+2] == "LIMIT":
                            limit_tok = "LIMIT"
                    elif i+1 < cur_len and sql_tokens[i+1] == "LIMIT":
                        limit_tok = "LIMIT"
                    
                    columns[tok].append(cur_nest + " " + cur_struct + " " + dasc_tok + " " + limit_tok)
        
            else:
                print("\n------------Warning: unexcepted COLUMN label format")
    
    column_labels = {}
    for col, labels in columns.items():
        label_str = " ".join([l.strip() for l in labels])
        column_labels[col] = label_str
        
    return column_labels


##### Populate one example for a given database based on a given nl-SQL template and sql component mapping

In [19]:
def populate_one(db, templates, templates_one, sql_components):
    """
    'P0=P1==', 'P0=P1=P2==', 'P0==', 'P1==', 'P0=>', 'P0=<', '{AGG0}=MAX', '{AGG0}=MIN'
    'T0-T1-JOIN', 'T0-T1-NO-JOIN', 
    'C0-id',, 'C2-id', , 'C1-id',  'C3-T1'
    """
    if len(db) > 1:
        template = random.choice(templates)
    else:
        template = random.choice(templates_one)
        
    sql_constraints = template['SQL constraints']
    sql_pattern = template["SQL pattern"]
    question, q_constraints = random.choice(template["questions"])
    constraints = list(set(sql_constraints + q_constraints))

    slots, columns, ops, vals, aggs, dasc = get_sql_slots(sql_pattern)
    slot_values, columns, skip_db_with_one_table = process_constraints(constraints, columns, slots)

    q_slots = get_q_slots(question)
    q_slot_values = {}   

    # 1 process ops - update columns and values constraints
    for op, colv in ops.items():
        if colv[0] == "*":
            if op not in slot_values.keys():
                op_val = random.choice([">", "<", ">=", "<=", "="])
                slot_values[op] = op_val
                if len(colv) == 2:
                    slot_values[colv[1]] = random.randint(1, 10)
        else:
            if colv[0] not in columns.keys():
                print("\n-----colv[0] not in columns.keys(): ")
                print(columns.keys())
                print(ops)
            assert colv[0] in columns.keys()
            if op not in slot_values.keys():
                if random.random() < 0.4:
                    op_val = "="
                else:
                    op_val = random.choice(OPS)
                slot_values[op] = op_val
                if op_val in [">", "<", ">=", "<="]:
                    columns[colv[0]].append("number")
            if len(colv) == 2:
                columns[colv[0]].append(colv[1])
    
    # 2 process columns
    random.shuffle(db)
    table_0, table_1 = None, None
    table_label_0 = ""
    table_label_1 = ""
    use_table_1 = False
    
    if "{COLUMN0}" in columns.keys() or "{TABLE0}" in q_slots:
        table_label_0 = "SELECT"
        
    if len(db) >= 2:
        table_0, table_1 = db[:2]
        if "{TABLE1}" in q_slots:
            table_label_1 = "SELECT"
            if "{TABLE0}" in q_slots:
                # p<0.5 from T0, T1 AND to SELECT T1 *
                # otherwise all from T0 AND to SELECT T1 *
                if random.random() < 0.5:
                    use_table_1 = True                 
            else:
                # p<0.4 all from T0 
                # AND to SELECT T1 *
                if random.random() < 0.6:
                    use_table_1 = True
                    if "{COLUMN1}" in columns.keys():
                        table_label_1 = "SELECT"
        else:
            # p<0.5 from T0, T1 AND to SELECT T1 *
            # otherwise all from T0, NOT to SELECT T1 *
            if random.random() < 0.5:
                use_table_1 = True
                if "{COLUMN1}" in columns.keys():
                    table_label_1 = "SELECT"
    else:
        table_0, table_1 = db[0], db[0]
    
    T0 = table_0["name"]
    T1 = table_1["name"]
    columns_inf_0 = list(zip(table_0["columns"], table_0["column_types"], table_0["values"]))[1:]
    if use_table_1:
        columns_inf_1 = list(zip(table_1["columns"], table_1["column_types"], table_1["values"]))[1:]
        
    if "{COLUMN0}" in columns.keys():
        col_0, value_0 = gen_col_info("{COLUMN0}", columns, columns_inf_0)
        slot_values["{COLUMN0}"] = col_0
        if value_0 is not None:
            for k, v in value_0:
                slot_values[k] = v
        if len(columns_inf_0) > 2:
            columns_inf_0 = [(col, ctype, val) for col, ctype, val in columns_inf_0 if col != col_0]

    if use_table_1:
        columns_input = columns_inf_1
    else:
        columns_input = columns_inf_0
            
    if "{COLUMN1}" in columns.keys():
        col_1, value_1 = gen_col_info("{COLUMN1}", columns, columns_input)
        slot_values["{COLUMN1}"] = col_1
        if value_1 is not None:
            for k, v in value_1:
                slot_values[k] = v
        columns_input_org = columns_input
        if len(columns_input) > 3:
            columns_input = [(col, ctype, val) for col, ctype, val in columns_input if col != col_1]
        if len(columns_input) < 2:
            columns_input = columns_input_org
        
    if "{COLUMN2}" in columns.keys():
        col_2, value_2 = gen_col_info("{COLUMN2}", columns, columns_input)
        slot_values["{COLUMN2}"] = col_2
        if value_2 is not None:
            for k, v in value_2:
                slot_values[k] = v
        columns_input_org = columns_input
        if len(columns_input) > 2:
            columns_input = [(col, ctype, val) for col, ctype, val in columns_input if col != col_2]
        if len(columns_input) < 2:
            columns_input = columns_input_org
                
    if "{COLUMN3}" in columns.keys():
        col_3, value_3 = gen_col_info("{COLUMN3}", columns, columns_input)
        slot_values["{COLUMN3}"] = col_3
        if value_3 is not None:
            for k, v in value_3:
                slot_values[k] = v
        
    # 3 aggs
    for agg in aggs.keys():
        if agg not in slot_values.keys():
            slot_values[agg] = random.choice(["MAX", "MIN", "SUM", "AVG"])
    # 4 values
    NUM = 1
    for val, cond in vals.items():
        assert val not in slot_values.keys()
        if cond == "integer":
            if random.random() < 0.5:
                slot_values[val] = 1
            else:
                NUM = random.randint(2, 10)
                slot_values[val] = NUM
        else:
            slot_values[val] = random.randint(0, 100)
                    
    # 5 dasc - true
    if dasc == True:
        slot_values["{DASC}"] = random.choice(["ASC", "DESC"])
    
    # 6 check if all sql slot values are done
    if len(slots) != len(slot_values):
        print("\nlen(slots) != len(slot_values)")
        print("sql_pattern: ", sql_pattern)
        print("slots: ", slots)
        print("slot_values: ", slot_values.keys())
    assert len(slots) == len(slot_values)
    
    # 7 for the questions slots:
    for qs in q_slots:
        if qs == "{TABLE0}":
            q_slot_values["{TABLE0}"] = T0
        elif qs == "{TABLE1}":
            q_slot_values["{TABLE1}"] = T1
        elif "SC" in qs:
            sc = slot_values["{DASC}"]
            if "SC" == qs:
                q_slot_values[qs] = random.choice(sql_components["SC"][sc])
            elif "SC_COL_LIMIT" == qs:
                if NUM > 1:
                    sc =  sc + "_NUM"
                    q_slot_values[qs] = random.choice(sql_components["SC_COL_LIMIT"][sc]).replace("[NUM]", str(NUM))
                else:
                    q_slot_values[qs] = random.choice(sql_components["SC_COL_LIMIT"][sc])
            elif "SC_COL_COUNT_LIMIT" in qs:
                sc_type = qs.replace("SC_COL_COUNT_LIMIT", "")
                if NUM > 1:
                    sc =  sc + "_NUM" + sc_type
                    q_slot_values[qs] = random.choice(sql_components["SC_COL_COUNT_LIMIT"][sc]).replace("[NUM]", str(NUM))
                else:
                    sc =  sc + sc_type
                    q_slot_values[qs] = random.choice(sql_components["SC_COL_COUNT_LIMIT"][sc])
            else:
                if "-" not in qs:
                    print("qs wrong", qs)
                assert "-" in qs
                if "C1" in qs:
                    sc_col = slot_values["{COLUMN1}"]
                elif "C2" in qs:
                    sc_col = slot_values["{COLUMN2}"]
                q_slot_values[qs] = random.choice(sql_components["SC_COL"][sc]).replace("[COL]", sc_col)
        else:
            if qs not in slot_values.keys():
                print("qs not in sv: ", qs)
                print("sql_pattern: ", sql_pattern)
                print("slot_values: ", slot_values)
            assert qs in slot_values.keys()
            if "OP" in qs:
                q_slot_values[qs] = random.choice(sql_components["OP"][slot_values[qs]])
            elif "AGG" in qs:
                q_slot_values[qs] = random.choice(sql_components["AGG"][slot_values[qs]])
            elif "COLUMN" in qs:
                q_slot_values[qs] = " ".join(slot_values[qs].split(" ")[1:6])
            elif "VALUE" in qs:
                q_slot_values[qs] = " ".join(str(slot_values[qs]).split(" ")[:5])
            else:
                print("\nWarning: some q slot type not considered!")
                print(qs)
    
    # 8 check if all question slots are processed
    assert len(q_slots) == len(q_slot_values)
    
    # 9 generate final SQL-question pair
    question_gen = replace_dict(question, q_slot_values)
    
    
    # 10 generate column labels
    slot_values_new = {}
    for sl, vl in slot_values.items():
        if "COLUMN" in sl:
            slot_values_new[sl] = "_=_".join(vl.split(" "))
        else:
            slot_values_new[sl] = vl
            
    column_labels = get_labels(sql_pattern)
    column_lables_real = {}
    for col, label in column_labels.items():
        if col != "*":
            col = slot_values[col]
        for slot, value in slot_values.items():
            label = label.replace(slot, str(value))
        column_lables_real[col] = label
    
    # also add labels for table column * 
    if table_label_0 != "":
        column_lables_real[table_0["columns"][0]] = table_label_0
    if table_label_1 != "":
        column_lables_real[table_1["columns"][0]] = table_label_1

    sql_gen = replace_dict(sql_pattern.replace(" {FROM}", ""), slot_values_new)
    
    return (sql_gen, question_gen, column_lables_real)

##### generatee examples for all databases

In [24]:
# let's start data augmentation!
def augment_db(db, templates, templates_one_table, sql_components, aug_limit):
    count = 0
    augment_pairs = []
    while count < aug_limit:
        sql_gen, question_gen, column_lables = populate_one(db, templates, templates_one_table, sql_components)
        augment_pairs.append((question_gen, sql_gen, column_lables))
        count += 1
    
    return augment_pairs
    

def augment_all_dbs(dbs, templates, templates_one_table, sql_components, aug_limit):
    augment_data = {}
    for idx, db in enumerate(dbs):
        if idx % 10000 == 0:
            print("processed: ", idx)
        db_cols = ["*"]
        db_values = [""]
        for tab in db:
            db_cols.extend(tab["columns"])
            db_values.extend(tab["values"])
        schema_str = " </s> ".join(db_cols)
        values_str = " </s> ".join([str(k) for k in db_values])
        schema_str = schema_str + " |-| " + values_str
        augment_pairs = augment_db(db, templates, templates_one_table, sql_components, aug_limit)
        augment_data[schema_str] = augment_pairs
    
    return augment_data

In [25]:
augment_data_webtable = augment_all_dbs(webtable_dbs, templates, templates_one_table, sql_components, 5)

processed:  0
processed:  10000
processed:  20000
processed:  30000
processed:  40000
processed:  50000
processed:  60000
processed:  70000
processed:  80000


##### Map SQL labels of all augmented examples into numeric labels

In [26]:
### process label prints for each column
def get_label_map(data):
    label_dict = defaultdict(int)
    for schema_str, example_list in data.items():
        for example in example_list:
            (question, sql, col_labels) = example
            for val in col_labels.values():
                label_dict[val] += 1
    label_list = sorted(label_dict.items(), key=lambda kv: kv[1], reverse=True)
    label_map = {}
    count = 1
    for label, _ in label_list:
        label_map[label] = count
        count += 1
    
    return label_map

def map_labels(data, label_map, is_dev=False):
    data_new = {}
    skip_count = 0
    count = 0
    for schema_str, exs in data.items():
        count += 1
        if count % 100000 == 0:
            print("processed: ", count)
        data_new[schema_str] = []
        for ex in exs:
            skip = False
            label_dict = ex[2]
            label_dict_new = {}
            for col, label in label_dict.items():
                if label in label_map.keys():
                    label_dict_new[col] = label_map[label]
                else:
                    skip = True
                    skip_count += 1
                    #else just skip
#             print(label_dict_new)
            if not skip:
                data_new[schema_str].append((ex[0], ex[1], ex[2], label_dict_new))   
    
    print("skip_count: ", skip_count)
    return data_new

In [27]:
import pickle
label_map_file = "data/labels_map.pkl"
# label_map_final = get_label_map(fine_tuning_data_augment_with_dev_wikisql)
# with open(label_map_file, 'wb') as fp:
#     pickle.dump(label_map_final, fp, protocol=pickle.HIGHEST_PROTOCOL)
with open(label_map_file, 'rb') as fp:
    label_map = pickle.load(fp)
    
augment_data_webtable = map_labels(augment_data_webtable, label_map)

skip_count:  0


##### Write and save file

In [29]:
MAX_TOKEN_LEN = 150
def write_final_file(augment_data):
    data_json = []
    skip_count = 0
    line_count = 0
    dup_count = 0
    pro_count = 0
    for schema_str, exs in augment_data.items():
        for ex in exs:
            line_count += 1
            if line_count % 100000 == 0:
                print("processed: ", line_count)
            question, sql, label_strs, label_ints = ex
            col_str, val_str = schema_str.split(" |-| ")
            colns = col_str.split(" </s> ")
            values = val_str.split(" </s> ")
            assert len(colns) == len(values)
            cols = []
            label_num = len(label_ints)
            label_count = 0
            for idx, coln in enumerate(colns):
                col = {}
                col["name"] = coln
                col["value"] = values[idx]
                if coln != "*":
                    col["name"] = " ".join(coln.split(" ")[1:])
                col["label_int"] = 0
                if coln in label_ints.keys():
                    col["label_int"] = label_ints[coln]
                    label_count += 1
                cols.append(col)
            
            assert label_count >= label_num
            if label_count > label_num:
                dup_count += 1
#                 print("\nWARNING: deplicated columns!")
#                 print("label_ints: ", label_ints)
#                 print("colns: ", colns)
            
            col_list = []
            label_list = []
            value_list = []
            col_count = 0
            for i, col in enumerate(cols):
                if col_count > 40 and col["label_int"] == 0:
                    continue
                col_list.append(col["name"])
                value_list.append(col["value"])
                col_count += 1
                label_list.append(int(col["label_int"]))
            assert len(col_list) == len(value_list)
            
            label_str = " ".join([str(k) for k in label_list])
            q_col_str = "<s> " + question.lower() + " </s> " + " </s> ".join(col_list).strip() + " </s> "
            caption = q_col_str + " ||| " + label_str
            tokens = tokenizer.tokenize(q_col_str)
            if len(tokens) > MAX_TOKEN_LEN:
                continue
                
            data_json.append({"question": question.lower(),
                              "columns": col_list,
                              "rows": [value_list],
                              "column_labels": label_list
                             })
            pro_count += 1

    print("total line: ", line_count)
    print("skiped line: ", skip_count)
    print("dup line: ", dup_count)
    print("pro line: ", pro_count)
    
    return data_json

In [30]:
data_json = write_final_file(augment_data_webtable)

processed:  100000
processed:  200000
processed:  300000
processed:  400000
total line:  418460
skiped line:  0
dup line:  2237
pro line:  413173


In [33]:
data_json[2000]

{'question': 'find the winner in which the in total prize is ignore $ 11,111.11 .',
 'columns': ['*',
  'online poker *',
  'date',
  'event #',
  'event',
  'winner',
  'prize',
  'entries',
  'prize pool',
  'elapsed time'],
 'rows': [['',
   'all',
   'may 1',
   '1l',
   "$ 11 no limit hold'em",
   'willrobrobu',
   '$ 11,111.11',
   '11,111',
   '$ 111,110',
   '[ 1-day ] event']],
 'column_labels': [0, 1, 0, 0, 0, 2, 175, 0, 0, 0]}

In [31]:
with open('wikitable_augment.json', 'w') as outfile:
    json.dump(data_json, outfile)